In [1]:
import json
import sqlite3
import pandas as pd
import config
import numpy as np
import requests

from sqlalchemy import create_engine
engine = create_engine('postgresql://'+config.db['user']+':'+config.db['password']+config.db['url'])
#f = open('./data/NVDRstock-trading31-01-23.json')
#data = json.load(f)
#data = pd.read_json('./data/NVDRstock-trading31-01-23.json')
sSource = "api"
if sSource=="file":
    with open('./data/Pricecomposition02-02-23.json','r',encoding='utf-8') as f:
        data = json.loads(f.read())
elif sSource=="api":
    url = 'https://www.set.or.th/api/set/index/set100/composition?lang=en'
    headers = {
        'Content-Type': 'application/json',
        'Host': 'www.set.or.th',
        'Origin': 'https://www.set.or.th',
        'Referer' : 'https://www.set.or.th/th/market/product/stock/search?market=set&index=SET100',
        'Cookie': 'id=KqKgOXZeJxapT8k8AL1rOg%3D%3D00000000;'
        }
    r = requests.get(url,headers=headers)
  
    data = r.json()
    if r.status_code!=200:
        exit

print(data['indexInfos'][0]['marketDateTime'][:10])
#cdate = pd.DataFrame(data).iloc[0].astype(str).str[:10]['date'], record_prefix=
#d1.info()
#print(cdate)
print(data)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
df = pd.json_normalize(data['composition']['stockInfos']) 
df.drop(['average', 'floor','ceiling', 'trVolume','trValue', 'aomVolume','aomValue', 'bids','offers'], axis=1, inplace=True)
df.rename(columns = {'symbol': 'series', 'last': 'close', 'percentChange': 'pchange','totalVolume': 'vol','totalValue': 'value'}, inplace = True)
df.drop(df.iloc[:, 11:],axis = 1, inplace=True)
df['trddate'] = data['indexInfos'][0]['marketDateTime'][:10] #pd.to_datetime('today').date() - pd.Timedelta("1 day")
print(df['trddate'][0])
a = [0]
df = df[~df['close'].isin(a)]
df.info()
print(df)
# df['composition.stockInfos'].info()
# print(df['composition.stockInfos'])

2023-07-20
<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   series   100 non-null    object 
 1   sign     100 non-null    object 
 2   prior    100 non-null    float64
 3   close    100 non-null    float64
 4   open     100 non-null    float64
 5   high     100 non-null    float64
 6   low      100 non-null    float64
 7   change   100 non-null    float64
 8   pchange  100 non-null    float64
 9   vol      100 non-null    float64
 10  value    100 non-null    float64
 11  trddate  100 non-null    object 
dtypes: float64(9), object(3)
memory usage: 10.2+ KB
    series sign   prior   close    open    high     low  change   pchange  \
0      AAV         2.70    2.64    2.70    2.70    2.62   -0.06 -2.222222   
1      ACE         1.89    1.88    1.90    1.91    1.88   -0.01 -0.529101   
2   ADVANC       221.00  220.00  221.00  222.00  219.00   -1.00 -0.45

In [11]:
print(df)
sql = "select * from sprice where trddate = '"+ df['trddate'].astype(str)[0] + "'"
print(sql)
rst = pd.read_sql(sql, con=engine)
print(len(rst.axes[0]))
if len(rst.axes[0]) == 0:df.to_sql(name='sprice', con=engine, if_exists='append')
p2 = pd.read_sql("select * from sprice where trddate = '"+ pd.to_datetime('today').strftime('%Y-%m-%d') + "'", con=engine)
print(p2)
len(df)

    series sign   prior   close    open    high     low  change   pchange  \
0      AAV         2.70    2.64    2.70    2.70    2.62   -0.06 -2.222222   
1      ACE         1.89    1.88    1.90    1.91    1.88   -0.01 -0.529101   
2   ADVANC       221.00  220.00  221.00  222.00  219.00   -1.00 -0.452489   
3    AMATA        23.50   23.30   23.50   23.90   23.30   -0.20 -0.851064   
4      AOT        71.75   70.50   71.50   71.75   70.50   -1.25 -1.742160   
..     ...  ...     ...     ...     ...     ...     ...     ...       ...   
95    TRUE         7.05    6.90    7.00    7.05    6.85   -0.15 -2.127660   
96     TTB         1.63    1.65    1.63    1.67    1.63    0.02  1.226994   
97      TU        12.90   12.80   12.90   13.00   12.80   -0.10 -0.775194   
98     VGI         2.88    2.82    2.86    2.90    2.82   -0.06 -2.083333   
99     WHA         4.94    4.90    4.96    4.98    4.86   -0.04 -0.809717   

            vol         value     trddate  
0    14317928.0  3.797760e+07  

100

In [8]:
#print(df)
df.rename(columns = {'date': 'trddate', 'buyValue': 'buy', 'sellValue': 'sell', 'netValue': 'net', 'totalValue': 'total', 'percentValue': 'percent', 'underlyingValue':'trdvalue'}, inplace = True)
df['ft']=df['total']/df['trdvalue']*100
df['ft'].round(2)
#df.info()
print(df)
#r.json()
#print(r.status_code)
#df = pd.json_normalize(r.json(), record_path =['nvdrTradings'])

#print(df)
df.rename(columns = {'date': 'trddate', 'buyValue': 'buy', 'sellValue': 'sell', 'netValue': 'net', 'totalValue': 'total', 'percentValue': 'percent', 'underlyingValue':'trdvalue'}, inplace = True)
df['ft']=df['total']/df['trdvalue']*100
df['ft'].round(2)
#df.info()
print(df)

KeyError: 'total'